머신 러닝 기술은 오프라인 어플리케이션이나 분석에 국한되지 않고 다양한 웹 서비스의 예측 엔진으로 사용할 수 있음

- 머신러닝 모델의 현재 훈련 상태 저장
- SQLite 데이터베이스를 사용하여 데이터 저장
- 유명한 플라스크 웹 프레임워크를 사용하여 웹 애플리케이션 개발
- 공개 웹 서버에 머신 러닝 어플리케이션 배포하기

# 1.학습된 사이킷런 추정기 저장

- 학습된 모델을 재사용하는 한 가지 방법은 파이썬의 `pickle` 모듈을 사용하는 것
- 이 모듈은 파이썬 객체의 구조를 압축된 바이트코드로 직렬화하고 복원할 수 있음
- 분류기의 현재 상태를 저장하고 레이블이 없는 새로운 샘플을 분류할 때 훈련 데이터에서 모델을 다시 학습할 필요 없이 저장된 모델을 불러오면 됨

---

```python
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)
```

- 위 코드는 앞선 감성분석의 외부 메모리 학습 방식으로 훈련한 로지스틱 회귀 모델을 저장

- 분류기 를 `pickle.dump`로 저장하고, NLTK의 불용어도 직렬화하여 저장한다면, NLTK 라이브러리를 설치할 필요가 없음
- `dump` 메서드의 파라미터
    - 첫 번째 매개변수: 대상 객체
    - 두 번째 매개변수: 파이썬 객체가 저장될 파일 객체
- `pickle` 모듈을 위해 `open` 함수에서 `wb` 매개변수를 사용하여 이진 모드로 파일을 엶
- `protocol=4`로 지정하여 파이썬 3.4 버전과 그 이상 버전에서 사용할 수 있는 가장 효율적이고 최신의 `pickl` 프로토콜을 사용
- `HashingVectorizer`는 학습 과정이 없기 대문에 `pickle`로 직렬화 할 필요가 없음
- 대신 현재 사용하고 있는 파이썬 세션에서 `HashingVectorizer` 객체를 임포트할 수 있도록 파이썬 스크립트를 만듦
```python
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower() + ' '.join(emoticons).replace('-', ''))
    tokenized=[w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
```

---

**joblib를 사용한 넘파이 배열 직렬화**

> - 로지스틱 모델에는 가중치 벡터 같은 여러 개의 넘파이 배열이 포함되어 있음 <br>
> - 넘파이 배열은 joblib 라이브러리를 사용하면 더 효율적으로 직렬화 할 수 있음

**pickle의 보안 문제**

> - pickle 모듈은 악성 코드에 대해 안전하지 않기 때문에 출처를 신뢰할 수 없는 곳에서 온 데이터를 복원하는 것은 잠재적으로 보안 위협이 있음 <br>
> - pickle은 어떤 객체라도 직렬화할 수 있으므로 복원 과정에서 pickle 파일에 저장된 악성 코드가 실행될 수 있음 <br>
> - 출처를 신뢰할 수 없는 곳으로부터 pickle 파일을 받았다면 특별히 주의를 기울이고 가상 환경이나 다른 사람이 접근할 수 없고 중요한 데이터가 저장되어 있지 않은 별도의 컴퓨터에서 복원하는 것이 좋음


In [1]:
import pickle
import re
import os
from etc_files.movieclassifier.vectorizer import vect

clf = pickle.load(open(os.path.join('./datasets/movieclassifier/pkl_objects', 'classifier.pkl'), 'rb'))

In [2]:
import numpy as np
label = {0: '음성', 1: '양성'}
example = ['I love this movie']
X = vect.transform(example)
print('예측: %s\n확률: %.2f%%' % (label[clf.predict(X)[0]], np.max(clf.predict_proba(X))*100))

예측: 양성
확률: 81.44%


# 2.데이터를 저장하기 위해 SQLite 데이터베이스 설정

- 사용자 피드백을 저장하기 위한 간단한 SQLite 데이터 베이스 세팅
- 이를 활용하여 분류 모델 업데이트
- SQLite는 오픈 소스 SQL 데이터베이스 엔진
- 별도의 서버가 필요하지 않아 작은 프로젝트나 간단한 웹 애플리케이션에 적합
- 기본적으로 SQLite 데이터베이스는 직접 접근할 수 있는 독립된 단일 데이터베이스 파일이라고 생각할 수 있음
- 시슽메에 특화된 설정이 없으며 대부분의 운영 체제를 지원
- SQLite는 `sqlite3` API에 포함되어 있음

In [3]:
import sqlite3

conn = sqlite3.connect('./datasets/movieclassifier/reviews.sqlite')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db' ' (review TEXT, sentiment INTEGER, date TEXT)')
example1='I love this movie'
c.execute("INSERT INTO review_db" " (review, sentiment, date) VALUES" " (?, ?, DATETIME('now'))", (example1, 1))
example2='I disliked this movie'
c.execute("INSERT INTO review_db" " (review, sentiment, date) VALUES" " (?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()

**코드 해석**

- `sqlite` 라이브러리의 `connect` 메서드를 호출하여 SQLite 데이터베이스 파일에 연결 (`conn`)
- `movieclassifier` 디렉토리에 `reviews.sqlite`가 없다면 새로운 데이터베이스 파일을 만듦
- 그다음 `cursor` 메서드로 데이터베이스 커서를 만듦
    - 커서를 통해 다양한 SQL 문법으로 데이터베이스 레코드를 조작할 수 있음
- 첫 번째 `execute` 메서드를 실행해서 새로운 데이터베이스 테이블 `review_db`를 생성
    - 이 테이블에 새로운 레코드를 추가하거나 읽을 수 있음
- `review_db` 테이블과 함께 `review, sentiment, date` 세 개의 컬럼도 생성
- 이 테이블에 두 개의 샘플 리뷰와 이에 상응하는 클래스 레이블(긍정/부정)을 저장
- `DATETIME('now')` SQL 명령은 래ㅔ코드 컬럼에 날짜와 시간을 추가
    - 물음표를 사용하여 영화 리뷰 텍스트(example1, example2)와 클래스 레이블 (1과0)로 이루어진 튜플의 원소를 `execute` 메서드의 매개변수에 순서대로 전달
- 마지막으로 `commit` 메서드를 사용하여 데이터베이스의 변경 사항을 저장하고 `close` 메서드로 데이터베이스와 연결을 닫음
- 다음으로 테이블에 제대로 저장되었는지 확인하기 위해 데이터베이스에 다시 연결하여 `SELECT` sQL 명령으로 2017년 시작부터 오늘가지 테이블에 추가된 모든 로우를 추출

In [4]:
conn = sqlite3.connect('./datasets/movieclassifier/reviews.sqlite')
c = conn.cursor()
c.execute(
    "SELECT * \
    FROM review_db \
    WHERE date"
    # "BETWEEN '2017-01-01 00:00:00' AND DATETIME('now')" 
)
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2023-06-18 14:06:02'), ('I disliked this movie', 0, '2023-06-18 14:06:02')]


# 3.플라스크 웹 애플리케이션 개발

- 링크드인과 핀터레스트에서 플라스크를 사용
- 플라스크는 파이썬으로 만들어졌기 때문에 영화 분류기처럼 파이썬 프로그래머가 기존 코드에 편리하게 통합할 수 있는 인터페이스를 제공

## 3.1.첫 번째 플라스크 애플리케이션

- 영화 분류기 애플리케이션을 구현하기 전에 플라스크 API에 대해 알아 볼 것
- 첫 번째 애플리케이션은 이름을 입력하는 폼 필드 하나만 가진 간단한 웹페이지로 구성
- 웹 애플리케이션으로 이름을 입력하면 애플리케이션은 새로운 웹 페이지를 화면에 출력할 것
- 플라스크 프레임워크 안에서 코드 간에 변수를 전달하고 값을 저장하는 방법을 배움

**app.py (dir: 1st_flask_app_1)**

1. 애플리케이션은 하나의 모듈로 실행됨
    - 매개변수 `__name__`으로 새로운 플라스크 인스턴스를 초기화
    - 이렇게 하면 플라스크는 현재 디렉터리와 같은 위치에서 HTML 템플릿 폴더 `templates`를 찾음
    - `__name__`은 파이썬의 특수한 변수로 모듈 이름이 담김
    - 파이썬 모듈을 스크립트로 실행하면 `__name__`의 값은 `'__main__'`이 됨
    - 플라스크는 모듈 이름이 `'__main__'`일 경우 현재 위치에서 템플릿 폴더를 찾음   
2. 그 다음 라우트 데코레이터(`@app.route('/')`)를 사용하여 특정 URL이 index 함수를 실행하도록 지정
3. 여기서 index 함수는 단순히 templates 폴더 아래에 있는 f`irst_app.html` HTML 파일을 화면에 출력
4. 마지막으로 이 스크립트가 파이썬 인터프리터에 의해 직접 실행될 때만 `run` 메서드를 사용하여 어플리케이션을 시작
    - 이를 위해 `if __name__='__main__'` 구문을 사용

```python
# app.py
from flask import Flask, render_template

app = Flask(__name__)
@app.route('/')
def index():
    return render_template('first_app.html')

if __name__=='__main__':
    app.run()
```

---

**first_app.html (dir: 1st_flask_app_1/templates)**

```html
# first_app.html
<!doctype html>
<html>
    <head>
        <title>첫 번째 애플리케이션</title>
    </head>
    <body>
        <div>와우, 첫 번째 플라스크 웹 애플리케이션입니다.</div>
    </body>
</html>
```

## 3.2.폼 검증과 화면 출력

- 플라스크 웹 애플리케이션을 확장해서 HTML 폼 요소를 추가
- `WTForms` 라이브러리를 사용하여 사용자로부터 데이터를 어떻게 수집하는지 볼 것

1. `index` 함수를 확장해서 `wtforms`의 `TextAreaField` 클래스를 사용하여 시작 웹 페이지에 테스트 필드를 추가
    - 이 클래스는 사용자의 입력 텍스트가 안전한지 아닌지를 자동으로 확인
2. 또 새로운 함수 `hello`를 정의하여 HTML 폼으로 전달된 내용을 검증한 후 `hello.html` 페이지를 출력
3. 여기서는 폼에 입력된 데이터를 HTTP 바디에 실어 서버로 전송하는 `POST` 메서드를 사용
    - `app.run` 메서드에 `debug=True` 매개변수를 설정해서 플라스크 디버거를 활성화
    - 이 옵션은 새로운 웹 애플리케이션을 개발할 때 도움이 많이 됨

```python
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)

class HelloForm(Form):
    sayhello = TextAreaField('', [validators.DataRequired()])

@app.route('/')
def index():
    form = HelloForm(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form = HelloForm(request.form)
    if request.method=='POST' and form.validate():
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    return render_template('first_app.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)
```

---

**진자2 템플릿 엔진을 사용하여 매크로 구현**

- 진자2 템플릿 엔진을 사용하여 `_formhelpers.html` 파일에 매크로를 작성
- 나중에 `first_app.html` 파일에서 텍스트 필드를 화면에 출력하기 위해 이 파일을 임포트할 것

```html
{% macro render_field(field) %}
    <dt>{{ field.label }}
    <dd>{{ field(**kwargs)|safe }}
    {% if field.errors %}
        <ul class=errors>
            {% for error in field.errors %}
            <li>{{ error }}</li>
            {% endfor %}
        </ul>
    {% endif %}
    </dd>
    </dt>
{% endmacro %}
```

---

**CSS로 스타일 추가**

- `style.css`를 만들어 HTML 문서의 스타일을 어떻게 바꿀 수 있는지 확인
- CSS 파일은 HTML `<body>` 요소의 폰트 크기를 2배로 키움
- 이파일은 static 서브 디렉토리에 저장
- 이 위치는 플라스크가 CSS와 같은 정적 파일을 찾는 기본 디렉토리

```CSS
body{
    font-size: 2em;
}
```

---

**first_app.html**

- 사용자가 이름을 입력할 수 있는 텍스트 폼을 출력
- 헤더 섹션에 CSS 파일을 추가
- 이 스타일 파일은 HTML `<body>`에 있는 모든 텍스트의 크기를 변경
- HTML `<body>`에는 `_formhelpers.html`의 폼 매크로를 임포트하고 `app.py`에 저장된 `sayhello` 폼을 화면에 출력
    - `app.py`의 `index` 함수에서 `HelloForm` 클래스의 객체 `form`을 키워드 매개변수 `form`으로 `render_template` 함수에 전달
    - 즉, `first_app.html` 파일의 `form.sayhello` 는 `TextAreaField` 클래스의 객체
- 또 텍스트 필드 항목을 서버에 제출할 수 있도록 같은 `<form>` 태그 안에 입력 버튼 추가

```html
<!doctype html>
<html>
    <head>
        <title>첫 번째 어플리케이션</title>
        <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    </head>
    <body>
        {% from "_formhelpers.html" import render_field %}
        <div>이름을 입력해 주세요</div>
        <form method=post action="/hello">
            <dl>
                {{ render_field(form.sayhello) }}
            </dl>
            <input type=submit value='이름 입력' name='submit_btn'>
        </form>
    </body>
</html>    
```

---

**결과 페이지 만들기**

- 마지막으로 `hello.html` 파일을 만들고 `hello` 함수의 `return` 문에서 `render_template('hello.html', name=name)`을 반환하여 화면에 출력
    - `first_app.html`에서 `<form>`태그의 `action` 속성을 `"/hello"`로 지정했으므로 이름 입력 버튼을 누르면 `@app.route('/hello')` 데코레이터가 지정된 `hello` 함수가 호출됨
- `app.py` 파일에 정의된 `hello` 함수는 사용자가 텍스트 필드에 입력한 텍스트를 출력함

```html
<!doctype html>
<html>
    <head>
        <title>첫 번째 어플리케이션</title>
        <link rel="stylesheet" href="{{ url_for('static', filename='style.css')}}">
    </head>
    <body>
        <div>{{ name }} 님 안녕하세요! </div>
    </body>
</html>
```

# 4.영화 리뷰 분류기를 웹 애플리케이션으로 만들기

## 4.1.파일과 폴더: 디렉토리 구조 살피기

```text
movieclassifier
├─ app.py
├─ pkl_objects
│  ├─ classifier.pkl
│  └─ stopwords.pkl
├─ reviews.sqlite
├─ static
│  └─ style.css
├─ templates
│  ├─ _formhelpers.html
│  ├─ results.html
│  ├─ reviewform.html
│  └─ thanks.html
└─ vectorizer.py
```

## 4.2.메인 어플리케이션 app.py 구현

```python

```